In [1]:
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML

X=pd.read_csv('../data/processed/X.csv')
Y=pd.read_csv('../data/processed/Y.csv')
X_predict=pd.read_csv('../data/processed/X_predict.csv')
df_predict=pd.read_csv('../data/processed/df_predict.csv')

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.1-internal" 2018-10-16; OpenJDK Runtime Environment (build 11.0.1-internal+0-adhoc..src); OpenJDK 64-Bit Server VM (build 11.0.1-internal+0-adhoc..src, mixed mode)
  Starting server from /home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpt0th1fiw
  JVM stdout: /tmp/tmpt0th1fiw/h2o_carpiero_started_from_python.out
  JVM stderr: /tmp/tmpt0th1fiw/h2o_carpiero_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_carpiero_4a2081
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.686 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
X_Y=pd.concat([X,Y], axis=1)


In [4]:
x= X.columns.to_list()
y = "price"
x

['carat',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'cut_int',
 'color_int',
 'clarity_int']

In [35]:
# H2OAutoML(max_models=20,sort_metric='RMSE') 513.54
# H2OAutoML(max_models=50,sort_metric='RMSE')	514.594
# H2OAutoML(max_models=100,max_runtime_secs=2000,sort_metric='RMSE') 513.786   
# H2OAutoML(max_models=200,sort_metric='RMSE')
# H2OAutoML(max_models=400,max_runtime_secs=1000,sort_metric='RMSE')  516.046
# H2OAutoML(max_models=50,max_runtime_secs=1000,sort_metric='RMSE') 512.375
# H2OAutoML(max_models=20,max_runtime_secs=1000,sort_metric='RMSE') 519,203

train_X_Y=h2o.H2OFrame(X_Y)
aml = H2OAutoML(max_models=50,max_runtime_secs=1000,sort_metric='RMSE')
aml.train(x=x, y=y, training_frame=train_X_Y)


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


In [36]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200911_182627,515.103,265331,265331,263.687,0.0915423
StackedEnsemble_BestOfFamily_AutoML_20200911_182627,515.759,266008,266008,263.158,0.0906673
GBM_3_AutoML_20200911_182627,518.994,269355,269355,267.537,0.093399
GBM_2_AutoML_20200911_182627,524.656,275264,275264,273.407,0.097571
GBM_4_AutoML_20200911_182627,526.604,277312,277312,268.309,0.091631
GBM_grid__1_AutoML_20200911_182627_model_1,528.085,278874,278874,266.183,0.087984
GBM_1_AutoML_20200911_182627,528.42,279227,279227,277.115,0.101476
XGBoost_grid__1_AutoML_20200911_182627_model_2,540.224,291842,291842,275.159,0.0918175
GBM_grid__1_AutoML_20200911_182627_model_3,542.836,294671,294671,289.84,0.11363
XGBoost_3_AutoML_20200911_182627,544.137,296085,296085,291.489,0.108901


In [29]:
predict_X=h2o.H2OFrame(X_predict)
predictions_competition= aml.leader.predict(predict_X)
predictions_competition=predictions_competition.as_data_frame()


Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [30]:

submission = pd.DataFrame({'id': df_predict['id'], 'price': predictions_competition['predict']})
submission['id'] = submission['id'].astype('int64')
# submission.loc[94,'price'] =20631
submission.to_csv(f'/home/carpiero/IRONHACK/kaggle/H2OAutoML(max_models=100,max_runtime_secs=2000,so_513_786.csv', index=False)



In [55]:
h2o.cluster().shutdown()

H2O session _sid_ba7f closed.


In [31]:
# H2OAutoML(max_models=20,max_runtime_secs=1000,sort_metric='RMSE')  511.791
# count    13485.000000
# mean      3947.534876
# std       3940.890099
# min        342.779315
# 25%        935.550702
# 50%       2449.032989
# 75%       5329.584411
# max      18128.114212
# Name: price, dtype: float64

# H2OAutoML(max_models=20,max_runtime_secs=1000,sort_metric='RMSE')  514.349
# count    13485.000000
# mean      3945.590390
# std       3957.416982
# min        332.303338
# 25%        929.447763
# 50%       2447.357701
# 75%       5315.242484
# max      18172.341788
# Name: price, dtype: float64

# H2OAutoML(max_models=20,sort_metric='RMSE') 513.54
# count    13485.000000
# mean      3947.669040
# std       3955.296976
# min      -3632.844472
# 25%        929.536782
# 50%       2447.944121
# 75%       5330.010441
# max      18360.233323
# Name: price, dtype: float64

# H2OAutoML(max_models=50,sort_metric='RMSE')	514.594
# count    13485.000000
# mean      3949.975678
# std       3954.971628
# min        342.490903
# 25%        926.883552
# 50%       2451.419051
# 75%       5328.877249
# max      18114.604913
# Name: price, dtype: float64

# H2OAutoML(max_models=100,max_runtime_secs=2000,sort_metric='RMSE') 513.786 
# count    13485.000000
# mean      3951.208307
# std       3959.686934
# min        345.780824
# 25%        926.090826
# 50%       2451.074238
# 75%       5335.157959
# max      18193.464176
# Name: price, dtype: float64


submission['price'].describe()

count    13485.000000
mean      3951.208307
std       3959.686934
min        345.780824
25%        926.090826
50%       2451.074238
75%       5335.157959
max      18193.464176
Name: price, dtype: float64